In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import praw
from newsapi import NewsApiClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Import finBert Model and test out functionality

In [2]:
# finBERT model from https://huggingface.co/ProsusAI/finbert
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Model configurations
print(model.config.id2label)

{0: 'positive', 1: 'negative', 2: 'neutral'}


In [3]:
# Example text list to test out the model
text_list = [
    "I think this stock is going to skyrocket!",
    "The market is looking bearish right now.",
    "Not sure if this quarter's earnings will be good.",
    "I'm very optimistic about the future of crypto!"
]

In [4]:
# Model outputs
inputs = tokenizer(text_list, padding = True, truncation = True, return_tensors = "pt")
outputs = model(**inputs)

# Convert outputs into sentiment probabilities
logits = outputs.logits
softmax = torch.nn.Softmax(dim = 1)
probs = softmax(logits)
probs

tensor([[0.0637, 0.5210, 0.4153],
        [0.0333, 0.7954, 0.1713],
        [0.5968, 0.2075, 0.1957],
        [0.2114, 0.0106, 0.7780]], grad_fn=<SoftmaxBackward0>)

## Application of finBERT Model on Reddit posts for Financial Sentiment Analysis

In [5]:
# Connect to Python Reddit API Wrapper, PRAW
client_id = "-4n_iUGICBtLQdxfiyhFew"
client_secret = "q6zTGbjcmYVqF6_lt7Aw1VMNcS6ltQ"

reddit = praw.Reddit(
    client_id = client_id,
    client_secret = client_secret,
    user_agent = "financial sentiment"
)

In [6]:
# View current subreddit post titles, body text, and link to each post
sub_reddit = reddit.subreddit("investing")

for submission in sub_reddit.hot(limit=3):
    print("Title:", submission.title)
    print("URL:", submission.url)
    print("Text:", submission.selftext)  # The body of a text post
    print("-----")

Title: Daily General Discussion and Advice Thread - April 10, 2025
URL: https://www.reddit.com/r/investing/comments/1jvtdz0/daily_general_discussion_and_advice_thread_april/
Text: Have a general question?  Want to offer some commentary on markets?  Maybe you would just like to throw out a neat fact that doesn't warrant a self post?  Feel free to post here! 

Please consider consulting our FAQ first - https://www.reddit.com/r/investing/wiki/faq
And our [side bar](https://www.reddit.com/r/investing/about/sidebar) also has useful resources.  

If you are new to investing - please refer to Wiki - [Getting Started](https://www.reddit.com/r/investing/wiki/index/gettingstarted/)

The reading list in the wiki has a list of books ranging from light reading to advanced topics depending on your knowledge level. Link here - [Reading List](https://www.reddit.com/r/investing/wiki/readinglist)

The media list in the wiki has a list of reputable podcasts and videos - [Podcasts and Videos](https://www.

In [7]:
# Function to extract sentiment probabilities using finBERT model, then calculate and return the sentiment score
def finbert_predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    probs = softmax(outputs.logits)
    positive_prob = probs[0, 0].item()
    negative_prob = probs[0, 1].item()
    neutral_prob = probs[0, 2].item()
    sentiment_score = positive_prob - negative_prob
    return sentiment_score

### Subreddit: r/investing Sentiment Analysis

In [8]:
# Pull reddit posts from subreddit: r/investing
subreddit_investing = reddit.subreddit("investing")

# Analyze each post and produce a sentiment score
data_investing = []

for submission in subreddit_investing.hot(limit=10):
    title = submission.title
    text = submission.selftext
    combined_text = title + "." + " " + text
    sentiment_score = finbert_predict(combined_text)
    data_investing.append([title, text, sentiment_score])

# Create a sentiment score tabel respective to each post
df_investing = pd.DataFrame(data_investing, columns=['Title', 'Text', 'Sentiment Score'])
df_investing

,Title,Text,Sentiment Score
0,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.005809
1,The Gap Between the Rich and Poor Just Widened...,We’ve just witnessed the most blatant market m...,-0.302611
2,How the hell are people saying their 401k are ...,I’ve been seeing so much fear across the board...,-0.553927
3,"The Market isn't a safe place for investments,...",The problem with this investment environment i...,-0.482349
4,"US Inflation rate eases to 2.4% in March, lowe...",[CNBC Article Link](https://www.cnbc.com/2025/...,-0.845709
5,Why are people vehemently against buying and h...,I'm fairly certain Buffet himself advises agai...,-0.154870
6,"First the rumor, then the news...","Monday, the market spikes 8% in 30 minutes on ...",0.023106
7,The Mar - a - Lago Accord,"According to some analysts, Trump’s global tra...",0.443062
8,Why did the 90 day pause cause the markets to ...,"If we think about it, nothing fundamentally ha...",-0.311822
9,Today is why you do not time the market,This sub has been dominated by doomers lately....,0.054995


### Subreddit: r/wallstreetbets Sentiment Analysis

In [9]:
# Pull reddit posts from subreddit: r/investing
subreddit_wallstreetbets = reddit.subreddit("wallstreetbets")

# Analyze each post and produce a sentiment score
data_wallstreetbets = []

for submission in subreddit_wallstreetbets.hot(limit=10):
    title = submission.title
    text = submission.selftext
    combined_text = title + "." + " " + text
    sentiment_score = finbert_predict(combined_text)
    data_wallstreetbets.append([title, text, sentiment_score])

# Create a sentiment score tabel respective to each post
df_wallstreetbets = pd.DataFrame(data_wallstreetbets, columns=['Title', 'Text', 'Sentiment Score'])
df_wallstreetbets

,Title,Text,Sentiment Score
0,"What Are Your Moves Tomorrow, April 11, 2025",This post contains content not supported on ol...,-0.058117
1,Weekly Earnings Thread 4/7 - 4/11,,-0.193473
2,The White House now says the total tariff rate...,,-0.004713
3,"EU, China start talks on lifting EU tariffs on...",[https://finance.yahoo.com/news/eu-china-start...,0.761748
4,Bessent doesn't rule out delisting Chinese stocks,,-0.017198
5,Up 32k this week out of sheer luck. I have no...,Skipped a screenshot of losses so that the pos...,0.128627
6,Apple is charting flights for 600 tons of iPho...,What episode of South Park is this? This doesn...,-0.037273
7,Just in: eu pauses tariffs🇪🇺,,-0.005864
8,"$10,000 all in for Tesla puts.",I’m going to ride this to $0 because I’m fking...,0.225220
9,I sold my Tesla puts… sorry‼️‼️,Thank you guys so much for the $7000 in the pa...,0.323268


## Application of finBERT Model using NewsAPI for Financial Sentiment Analysis

In [11]:
# Initialize the NewsAPI with API Key
newsapi = NewsApiClient(api_key='799c5d4d17954b1799384ea22795a32b')

# Specifications for fetching news 
max_pages = 1         # number of pages to fetch 
page_size = 100       # how many articles per page 
query = "stock"       # keyword to search

# Send specifications to the NewsAPI to fetch news articles
response = newsapi.get_everything(
    q=query,
    language="en",
    page_size=page_size,
    page=max_pages,
)
    
# Extract articles from the response and store data
articles = response.get("articles", [])

all_articles = []
all_articles.extend(articles)

print(f"Total articles fetched: {len(all_articles)}")

Total articles fetched: 97


In [12]:
# Analyze each news article and produce a sentiment score
data_news = []

for article in all_articles:
    title = article.get('title')
    description = article.get('description')
    combined_text = f"{article.get('title', '')} {article.get('description', '')}"
    sentiment_score = finbert_predict(combined_text)
    data_news.append([title, description, sentiment_score])
    
# Create pandas dataframe to store each news article and their sentiment scores    
df_news = pd.DataFrame(data_news, columns=['Title', 'Description', 'Sentiment Score'])
df_news

,Title,Description,Sentiment Score
0,Chinese Companies Rush to Put DeepSeek in Ever...,From video game developers to a nuclear power ...,0.637322
1,Apple has its biggest stock drop in five years...,"Shares of Apple, Amazon and other tech stocks ...",-0.966677
2,Apple has its biggest stock drop in five years...,"Shares of Apple, Amazon, and other tech stocks...",-0.965345
3,Instacart will pay shoppers to take videos of ...,Instacart is introducing a new way for shopper...,0.139486
4,Tesla Hate Is Making Insurance More Expensive ...,As stock slumps and Americans turn against the...,0.047349
...,...,...,...
92,Top 3 Dividend Stocks To Enhance Your Portfolio,As the U.S. stock market grapples with volatil...,0.800693
93,Is Exxon Mobil (XOM) the Most Undervalued S&P ...,None,0.018797
94,Tesla beware: Rival BYD unveils ultra-fast cha...,None,0.864068
95,"Tesla stock bleeds, recession fears — and Bitc...",None,-0.345979
